##### Исполнение итогового задания по Проекту 3 "О вкусной и здоровой пище". <br>
  Автор -  <b>Петроченков Виктор Александрович</b><br>
  Группа SF-DST-27<br>
  Юнит 3. Введение в машинное обучение 

##### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

import regex as re
from dateutil import parser

import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns 

# Для работы со словарём
from collections import OrderedDict
from operator import itemgetter

# Инструмент для разделения датасета
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [2]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# Зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
RANDOM_SEED = 89
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('14/11/2020')

## Импорт данных


In [5]:
df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')
sample_submission = pd.read_csv('sample_submission.csv')
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data = pd.DataFrame(data)

In [ ]:
data.info()

In [ ]:
# Выводим процентный список пропущенных данных
print('Процентный список пропущенных данных')
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

#### Признаки:

<b>City</b>: Город<br>
<b>Cuisine Style</b>: Тип кухни<br>
<b>Ranking</b>: Ранг ресторана относительно других ресторанов в этом городе<br>
<b>Price Range</b>: Цены в ресторане в 3 категориях<br>
<b>Number of Reviews</b>: Количество отзывов<br>
<b>Reviews</b>: 2 последних отзыва и даты этих отзывов<br>
<b>URL_TA</b>: страница ресторана на 'www.tripadvisor.com'<br>
<b>ID_TA</b>: ID ресторана в TripAdvisor<br>
<b>Rating</b>: Рейтинг ресторана<br>

In [ ]:
# Выведем количество уникальных значений по столбцам
data.nunique(dropna=False)

Из полученных данных по количеству уникальных значений, можно сделать выводы:
- индикационные номера ресторанов повторяются, это говорит о том, что один и тот же ресторан имеет разные значения признаков.
- рестораны представленные в датасете находятся в тридцати одном Европейских городах

In [ ]:
print('Количество пропусков: ', data['Restaurant_id'].isnull().sum())
print('Количество уникальных значений: ', data['Restaurant_id'].nunique())

In [ ]:
# Визуализируем один из ресторанов, который встречается в датасете самое большое количество раз
data.query('Restaurant_id == "id_871"')

Видим, что ресторан с id_871 представлен в 19 европейских городах и имеет достаточно высокий рейтинг.

Рассмотрим ID_TA.

In [ ]:
print(f'Отстутствующих данных в столбце ID_TA: {data.ID_TA.isnull().sum()}')
print(f'Уникальных записей в столбце ID_TA: {data.ID_TA.nunique()}\n'
      f'Неуникальных записей в столбце ID_TA: {data.ID_TA.count() - data.ID_TA.nunique()}')

Этот признак внутренний учётный параметр TripAdvisor и на рейтинг влияния - напрямую не оказывает. За этим параметром на сайте TripAdvisor можно получить дополнительную информацию. Неуникальные записи видимо ошибка учёта.

Столбец Cuisine Style на 23% не заполнен. Очевидно, что если заполнить пустые ячейки этого столбца типом European, то вероятность того, что в этих европейских ресторанах не подают блюда из европейской кухни ничтожна мала.

In [ ]:
# Заполнение пустых ячеек столбца  cuisine_style значением European
data['Cuisine Style'] = data['Cuisine Style'].fillna("['European']")

In [ ]:
# Выведем данные столбца
print('Количество пропусков: ', data['Price Range'].isnull().sum())
print('Вид уникальных значений: ', data['Price Range'].unique())

    Значения столбца имеют вид знака доллара, в виде его количества. Этот способ информирования говорит об разрядности ценового диапазона. Один знак $ обозначает, что можно поесть до 99$, два знака $$ до 999$, диапазон $$ - $$$ даёт знать, что покушать хорошо можно от 100 до 1000 $, в самые дорогие рестораны заходить покушать хорошо, без актива на кредитной карте меньше, чем 10000$ не стоит.
    Пустые ячейки заполним $$ (т.е. если у Вас есть евро в пересчёте на 1000$), то в даже очень дорогом ресторане вам удастся перекусить пиццей.

In [ ]:
# Заполним пустые ячейки столбца price_range значение редним значением цнового диапазона $$, считая что за максимум 100 $
# на одного достаточно.
data['Price Range'] = data['Price Range'].fillna('$$')

In [ ]:
# На этом этапе пустых ячеек столбца Number of Reviews 6%, заполним пустые ячейки нулём.
data['Number of Reviews'] = data['Number of Reviews'].fillna(0)

In [ ]:
print('Процентный список пропущенных данных')
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

##### Пропушенных данных в исходном датасете нет. Имеются 20% незаполненных данных в столбце Rating - это целевой показатель, его корректировать не будем.

## Визуализируем некоторые признаки датасета

In [ ]:
plt.rcParams['figure.figsize'] = (10,6)
# График - количество ресторанов в городах
data['City'].value_counts().plot(kind='barh')

# Получаем полный список городов представленных в датасете
city = dict(data['City'].value_counts())
city_pop = city.copy()
print(city)

Основное количество ресторанов расположены в Великобритании, Франции и Испании.

In [ ]:
# График количества ресторанов в зависимости от ранга
data['Ranking'].hist(bins=20)
print('Максимальный ранг: ',data['Ranking'].max(),'  Минимальный ранг: ',data['Ranking'].min())

Видим, что основное количестов ресторанов имею ранг 2500.
#### Визуализируем признак Rating по отношению к городам

In [ ]:
# Гистограмма распределения занчения Ranking ресторанов находящихся в городе Лондон.
data['Ranking'][data['City'] =='London'].hist(bins=20)

Распределение нормальное.Проверим по другим городам.

In [ ]:
# посмотрим на топ 15 городов
for x in (data['City'].value_counts())[0:15].index:
    data['Ranking'][data['City'] == x].hist(bins=20)
plt.show()

Видим, что независимо от городов сохраняется нормальное распределение признака Ranking.

In [ ]:
# График распределения целевой переменной
data['Rating'].value_counts(ascending=True).plot(kind='barh')
print('Максимальный рейтинг: ',data['Rating'].max(),'  Минимальный рейтинг: ',data['Rating'].min())

Датасет представленных ресторанов в основном с хорошим рейтингом, основное количество ресторанов имеют рейтинг не ниже 3 (максимальный рейтинг 5).

In [ ]:
# Проверяем датасет на наличие дубликатов строк
data[data.duplicated()]

#### Дубликатов строк нет

### Проверяем столбцы датасета на наличие выбросов

In [ ]:
# Формируем список наименований столбцов имеющие числовой тип данных
list_col =[]
for x in data.columns:
    if data[x].dtype == 'float64' or data[x].dtype == 'int64' or data[x].dtype == 'uint8':
        list_col.append(x)
list_col

In [ ]:
# Расчёт выбросов по столбцам
for y in list_col:
    Q1_date = data[y].quantile(.25)
    Q3_date = data[y].quantile(.75)
    q1_date = Q1_date-1.5*(Q3_date-Q1_date)
    q3_date = Q3_date+1.5*(Q3_date-Q1_date)
    min_q = [x for x in data[y] if x < q1_date]
    max_q = [x for x in data[y] if x > q3_date]
    if len(min_q) == 0 and len(max_q):
        print('Столбец ', y, ' выбросов не имеет')
    else:
        print('Столбец ', y, ' имеет', len(min_q) + len(max_q), 'выбросов')    

### Выбросы присутствуют только в целевом показателе Rating  и в функциональном показателе sample !!!

In [ ]:
sns.pairplot(data, kind = 'reg')
data.corr()

Корреляционная матрица не содержит сильно коррелирующих признаков. Следовательно, все признаки сохраняются в датасете.
Отдельно хочется отметить на обратную корреляцию целевого показателя ранга Ranking и рейтинга Rating. Т.е. чем выше рейтинг, тем ниже ранг. Видимо учётные данные в TripAdvisor отранжированны таким образом, что рейтинговые рестораны имеют низкий ранг, т.е. стоят первые по списку.
#### <i>Примечание</i>: надо проверить

## Очистка, преобразование данных и генерация новых признаков

#### Признак - плотность населения population_density
Данные взяты из источников:

https://ru.qaz.wiki/wiki/List_of_European_Union_cities_proper_by_population_density

https://wiki2.org/ru/Список_самых_населённых_городов_Европейского_союза

Целесообразность ввода этого признака объясняется более высокой вероятностью посещения ресторанов при высокой плотности населения. Видимо следует ожидать влияния этого признака на целевой показатель.


In [ ]:
# Население городов (информация из Интернет)
population_density = {'London': 5082, 'Paris': 20909, 'Madrid': 5390, 'Barcelona': 15991, 'Berlin': 4090, 'Milan': 7519, 'Rome': 3389, 'Prague': 2473, 'Lisbon': 6446, 'Vienna': 4556, 'Amsterdam': 5135, 'Brussels': 7465, 'Hamburg': 2388, 'Munich': 4736, 'Lyon': 10460, 'Stockholm': 5012, 'Budapest': 3337, 'Warsaw': 3421, 'Dublin': 5695, 'Copenhagen': 6711, 'Athens': 19135, 'Edinburgh': 4140, 'Zurich': 4666, 'Oporto': 6900, 'Geneva': 12589, 'Krakow': 2313, 'Oslo': 1483, 'Helsinki': 3035, 'Bratislava': 1189, 'Luxembourg': 2240, 'Ljubljana': 1583}
print(population_density)

In [ ]:
# Формируем столбец population_density
for y in range(0,len(data)):
    data.at[y,'population_density'] = population_density[data['City'][y]]

#### Признак - доля населения города приходящаяся на один ресторан population

In [ ]:
# Население городов (информация из Интернет)
population = {'London': 8416999, 'Paris': 2257981, 'Madrid': 3273049, 'Barcelona': 1619337, 'Berlin': 3479240, 'Milan': 1324110, 'Rome': 2761477, 'Prague': 1290211, 'Lisbon': 506654, 'Vienna': 1714142, 'Amsterdam': 779808, 'Brussels': 1125728, 'Hamburg': 1787220, 'Munich': 1353168, 'Lyon': 516092, 'Stockholm': 847073, 'Budapest': 1721556, 'Warsaw': 1716855, 'Dublin': 1173179, 'Copenhagen': 794128, 'Athens': 664046, 'Edinburgh': 513210, 'Zurich': 434008, 'Oporto': 237591, 'Geneva': 201818, 'Krakow': 755546, 'Oslo': 599230, 'Helsinki': 588549, 'Bratislava': 437725, 'Luxembourg': 626108, 'Ljubljana': 284355}
print(population)

In [ ]:
# Формируем новый признак - доля населения приходящаяся на один ресторан из датасета в каждом городе
# city - словарь в котором хранятся данные города - ключ и количество ресторанов в этом городе - значение.
for y in city:
    city_pop[y]=round(population[y]/city[y])

    
sort_dic = OrderedDict(sorted(city_pop.items(), key = itemgetter(1), reverse = False)) # Сортируем по значению и
# и формируем ранжированный список городов от меньшего к большему
print(sort_dic)

# Создаём новый столбец population
for y in range(0,len(data)):
    data.at[y,'population'] = city_pop[data['City'][y]]
    
# Пример. В Лондоне, на каждые 1462 лондонца приходится один ресторан из датасета. В Париже - на каждые 461 парижанина
# приходится один ресторан. Самый "обеспеченный" ресторанами город - Лиссабон, а видимо в Люксембурге столик в ресторане
# нужно заказывать заранее :)

#### Признак - охвата

Так как ресторан может принадлежать некой сети общепита, которая представлена в разных городах то уместно ввести интегрированный список долей по каждому городу относящемуся к конкретному ресторану для каждого Restaurant_id

In [ ]:
reach = dict(data.groupby('Restaurant_id')['population'].median())
for y in range(0,len(data)):
    data.at[y,'reach'] = round(reach[data['Restaurant_id'][y]])

#### Признак - индекс уровня жизни

Данные взяты с сайта https://www.numbeo.com/ по каждому городу - индекс уровня жизни.
При расчёте индекса уровня жизни использует девять факторов качества жизни для определения оценки страны/города. Показателей, которые учитывались при составлении рейтинга:<br>

•	Здоровье: Ожидаемая продолжительности жизни (в годах). Источник: US Census Bureau.<br>
•	Семейная жизнь: Уровень разводов (на 1 тыс. чел.), ставится оценка от 1 (мало разводов) до 5 (много разводов). Источники: ООН; Euromonitor.<br>
•	Общественная жизнь: Переменная принимает значение 1, если в стране высокий уровень посещаемости церкви или профсоюзного членства. Источник: World Values Survey.<br>
•	Материальное благополучие: ВВП на душу населения, Паритет покупательной способности. Источник: Economist Intelligence Unit.<br>
•	Политическая стабильность и безопасность: Рейтинги политической стабильности и безопасности. Источник: Economist Intelligence Unit.<br>
•	Климат и география: Широта, для различения холодных и жарких климатов. Источник: CIA World Factbook.<br>
•	Гарантия работы: Уровень безработицы (в процентах). Источник: Economist Intelligence Unit.<br>
•	Политическая свобода: Средний индекс политической и гражданской свободы. Шкала от 1 (полностью свободная) до 7 (несвободная). Источник: Freedom House.<br>
•	Гендерное равенство: измеряется путём деления средней зарплаты мужчин на зарплату женщин. Источник: UNDP Human Development Report.<br>
Данный показатель как признак необходимо включить в модель в силу того, что он характеризует уровень спроса на ресторанные услуги.


In [ ]:
# Индекс уровня жизни по городам (информация из Интернет)
living_index = {'London': 116.23, 'Paris': 136.37, 'Madrid': 64.68, 'Barcelona': 67.54, 'Berlin': 80.74, 'Milan': 111.36, 'Rome': 100.05, 'Prague': 31.53, 'Lisbon': 38.36, 'Vienna': 81.52, 'Amsterdam': 120.07, 'Brussels': 97.72, 'Hamburg': 86.39, 'Munich': 106.45, 'Lyon': 107.53, 'Stockholm': 118.49, 'Budapest':  17.09, 'Warsaw': 17.83, 'Dublin': 121.47, 'Copenhagen': 144.70, 'Athens': 58.58, 'Edinburgh': 89.91, 'Zurich': 244.96, 'Oporto': 36.68, 'Geneva': 233.66, 'Krakow': 9.94, 'Oslo': 68.34, 'Helsinki': 103.54, 'Bratislava': 34.47, 'Luxembourg': 127.85, 'Ljubljana': 55.48}
# Создаём новый столбец living_index
for y in range(0,len(data)):
    data.at[y,'living_index'] = living_index[data['City'][y]]
    
data['living_index'] = pd.to_numeric(data['living_index']) # Защита. Преобразуем значения столбца в число

#### Признак - стоимость на еду food_prices
Данные взяты с сайта https://www.numbeo.com/ по каждому городу. Признак представлен в ценнах USD, как минимальная сумма необходимая на одного человека для получения 2400 колорий в 31 день каждого месяца.

In [ ]:
food_prices = {'London':271.12, 'Paris':456.98, 'Madrid':263.60, 'Barcelona':269.14, 'Berlin':289.81, 'Milan':370.25, 'Rome':347.61, 'Prague':219.09, 'Lisbon':206.17, 'Vienna':315.33, 'Amsterdam':319.22, 'Brussels':330.92, 'Hamburg':282.96, 'Munich':331.71, 'Lyon':375.47, 'Stockholm':372.68, 'Budapest':184.50, 'Warsaw':181.17, 'Dublin':305.68, 'Copenhagen':373.24, 'Athens':235.71, 'Edinburgh':247.17, 'Zurich':731.17, 'Oporto':215.64, 'Geneva':704.94, 'Krakow':167.61, 'Oslo':448.58, 'Helsinki':321.60, 'Bratislava':224.90, 'Luxembourg':420.21, 'Ljubljana':262.88}
# Создаём новый столбец food_prices
for y in range(0,len(data)):
    data.at[y,'food_prices'] = food_prices[data['City'][y]]
    
data['food_prices'] = pd.to_numeric(data['food_prices']) # Защита. Преобразуем значения столбца в число

#### Признак - индекс преступности crime_index
Так как посещение ресторанов происходит в основном в вечернее время, а в это время "преступные элементы" активизируются, то вполне ожидаемо, что чем выше преступность в регионе, тем люди больше сидят дома и не ходят в рестораны. Считаю наличие этого признака вполне уместно.
Данные взяты с сайта https://www.numbeo.com/ по каждому городу. 
Индекс преступности —  в криминологии статистический показатель масштабов преступности в обществе, отражающий отношение количества совершенных и зарегистрированных преступлений к определенному количеству лиц.

In [ ]:
crime_index = {'London':52.65, 'Paris':52.50, 'Madrid':29.46, 'Barcelona':45.44, 'Berlin':40.79, 'Milan':43.27, 'Rome':51.69, 'Prague':24.05, 'Lisbon':28.18, 'Vienna':24.96, 'Amsterdam':33.02, 'Brussels':50.91, 'Hamburg':43.38, 'Munich':17.67, 'Lyon':44.72, 'Stockholm':45.17, 'Budapest':36.06, 'Warsaw':25.50, 'Dublin':49.67, 'Copenhagen':25.53, 'Athens':51.52, 'Edinburgh':30.96, 'Zurich':16.41, 'Oporto':36.28, 'Geneva':27.41, 'Krakow':25.68, 'Oslo':35.61, 'Helsinki':23.82, 'Bratislava':30.71, 'Luxembourg':27.79, 'Ljubljana':21.38}
# Создаём новый столбец crime_index
for y in range(0,len(data)):
    data.at[y,'crime_index'] = crime_index[data['City'][y]]
    
data['crime_index'] = pd.to_numeric(data['crime_index']) # Защита. Преобразуем значения столбца в число

#### Признак - индекс времени time_index

Посещение ресторана (финальная его стадия) связана с убытием домой и скорей всего на транспорте. Вполне логично включить в признак - индекс времени как среднее время, необходимое для перевозки в одну сторону, в минутах. Потенциальные клиенты ресторанов охотнее будут посещать рестораны, как ближайшие, так и удалённые, если индекс времени их устраивает, т.е. относительно мал.
Данные взяты с сайта https://www.numbeo.com/ по каждому городу. 

In [ ]:
time_index = {'London':43.81, 'Paris':41.28, 'Madrid':35.05, 'Barcelona':29.94, 'Berlin':33.66, 'Milan':36.74, 'Rome':42.07, 'Prague':32.43, 'Lisbon':35.79, 'Vienna':26.71, 'Amsterdam':29.62, 'Brussels':37.17, 'Hamburg':34.39, 'Munich':33.06, 'Lyon':33.78, 'Stockholm':36.09, 'Budapest':39.10, 'Warsaw':35.08, 'Dublin':40.25, 'Copenhagen':28.77, 'Athens':38.17, 'Edinburgh':28.85, 'Zurich':33.53, 'Oporto':28.59, 'Geneva':25.49, 'Krakow':30.71, 'Oslo':31.92, 'Helsinki':30.97, 'Bratislava':30.89, 'Luxembourg':33.24, 'Ljubljana':27.72}
# Создаём новый столбец time_index
for y in range(0,len(data)):
    data.at[y,'time_index'] = time_index[data['City'][y]]
    
data['time_index'] = pd.to_numeric(data['time_index']) # Защита. Преобразуем значения столбца в число

##### Признак - туристической мощности города tourist_people.

Очевидно, что в городах, где развит туризм, а таковые в датасете все, наполняемость ресторанов обеспечивает не только местное население но въездной туризм. Введём признак tourist_people.
Источник данных: 
 - https://zen.yandex.ru/media/podborka/50-samyh-posescaemyh-gorodov-mira-5cebbe6a99144700b4c12577
 - https://www.newsli.ru/news/world/turizm/34353
 
 Смысловое понимание этого признака следующее. Признак получается путём умножения населения города на количество туристов, посетивших этот город в течение года, и на коэффициент link. Если представить матрицу размером n x m, где n - количество жителей города, а m - количество туристов, то количество ячеек этой матрицы будет вычисляемое произведение, т.е. количество контактов каждого туриста с каждым жителем. Коэффициентом link можно регулировать количеством контактов принимая его значение от 0 до 1. В данном расчёте принимаем максимально возможную ситуацию, когда каждый горожанин встретился с каждым туристом и порекомендовал посетить тот или иной ресторан.:)
 <i>Примечание:</i> Многие данные приходилось собирать из разных источников и видимо временные границы размыты (т.е. разные годы). Обратил внимание, что сводной статистики, детализированной, в открытом доступе по направлению туризм, мало.


In [ ]:
#  Словарь содержащий информацию о количестве туристов посетивших конкретный город из датасете за год
tourist_people = {'London': 19827.8, 'Paris': 15834.2, 'Madrid': 5275.2, 'Barcelona': 6530.1, 'Berlin': 5627.4, 'Milan': 6347.9, 'Rome': 9531.6, 'Prague': 8806.7, 'Lisbon': 3639, 'Vienna': 6067.1, 'Amsterdam': 7848, 'Brussels': 4276, 'Hamburg': 8000, 'Munich': 4205, 'Lyon': 1500, 'Stockholm': 2721, 'Budapest': 4025, 'Warsaw': 2827, 'Dublin': 4986.5, 'Copenhagen': 3185, 'Athens': 4797.4, 'Edinburgh': 2320, 'Zurich': 1000, 'Oporto': 2491, 'Geneva': 3323, 'Krakow': 2910, 'Oslo': 4800, 'Helsinki': 3400, 'Bratislava': 1400, 'Luxembourg': 1018, 'Ljubljana': 1548}

# Создаём новый столбец tourist_people и заполним его промежуточным значениями количеством туристов кпо каждому городу
link = 1
for y in range(0,len(data)):
    data.at[y,'tourist_people'] = tourist_people[data['City'][y]]*population[data['City'][y]]*link
    
data['tourist_people'] = pd.to_numeric(data['tourist_people']) # Защита. Преобразуем значения столбца в число

## №1
В загруженном датасете проведём корректировку наименований столбцов

In [ ]:
# Нормализуем наименование столбцов согасно стандарта написания кода и заменим старые наименования на новые
data.columns = [x.replace('\n','_').replace(' ','_').replace('__','_').replace('-','_').replace('(','').replace(')','').lower() for x in list(data)]
data.columns

#### Наименования столбцов:

<b>restaurant_id</b> - ID ресторана<br>
<b>city</b> - город локации ресторана<br>
<b>cuisine_style</b> - стиль кухни<br>
<b>ranking</b> - ранжирование<br>
<b>rating</b> - рейтинг<br>
<b>population_density</b> - плотность населения города<br>
<b>price_range</b> - ценовой диапазон<br>
<b>number_of_reviews</b> - количество отзывов<br>
<b>reviews</b> - обзоры<br>
<b>url_ta</b> - url адрес ресторана в сети Интернет<br>
<b>id_ta</b> - идентификатор ресторана в базе данных TripAdvisor<br>
<b>population</b> - доля населения приходящаяся на один ресторан в каждом конретном городе<br>
<b>reach</b> - охват сетью ресторанов городов<br>
<b>living_index</b> - индекс уровня жизни в городах<br>
<b>food_prices</b> - сумма необходимая на одного человека для получения 2400 колорий в 31 день каждого месяца<br>
<b>crime_index</b> - индекс преступности в городах<br>
<b>time_index</b> - индекс времени, как среднее время, необходимое для перевозки в одну сторону, в минутах<br>
<b>tourist_people</b> - показатель туристической мощности города

## №2
В связи с тем, что стили кухонь в каждом ресторане предлагаются разные, и часто комплексно, проведём разнесение стилей кухонь по каждому ресторану в виде отдельных столбцов.

In [ ]:
pattern = re.compile('[^a-zA-Z&, ]+')
# Перевод значений столбца cuisine_style в строку и замена старого значения ячейки на список типов кухонь.
for x in range(0, len(data)):
    z = ''.join(data['cuisine_style'][x])
    z = re.sub(pattern,'',data['cuisine_style'][x])
    z = z.split(',')
    data.at[x,'cuisine_style'] = sorted([j.strip().replace('  ',' ').replace(' ','_') for j in z])

#### Признак - type_cuisine количество типов кухонь представляемых рестораном клиентам.

In [ ]:
# Определяем по каждому ресторану стили кухонь, добавляем к датасету новые столбцы, названия которых соотвествуют тому
# или иному стилю кухни и заполняем ячейки этих столбцов 1, в зависимости от наличия в ресторане стиля кухни
cuisine = [] # Список наименований столбцов для разных типов кухонь датасета
data['types_cuisine'] = 0 # Создаём столбец type_cuisine, с начальным значением равным 0
for x in range(0, len(data)):
    z = data['cuisine_style'][x]
    data['types_cuisine'][x] = len(z)
    for y in z:
        data.at[x,y] = 1
        if cuisine.count(y) == 0:
            cuisine.append(y)
            
for y in cuisine: # Заполняем пустые ячейки столбцов типов кухонь 0
    data[y] = data[y].fillna(0)
            
data = data.drop(['cuisine_style'], axis = 1) # Удаляем столбец cuisine_style

In [ ]:
dic_cuisine = {y:0 for y in cuisine} # Создаём словарь типов кухонь, в значения которых будут записаны количество раз 
# встречающиеся в датасете
for y in cuisine:
    dic_cuisine[y] = data[y].sum()
sort_dic = list(OrderedDict(sorted(dic_cuisine.items(), key = itemgetter(1), reverse = False))) # Сортируем по значению и
# и формируем ранжированный список типов кухонь от меньшего к большему
print(sort_dic)

#### Признак - top_cuisine сумарное значение ранжировок из  sort_dic представленных кухонь, в каждом конкретно взятом ресторане.

In [ ]:
data['top_cuisine'] = 0
data['top_cuisine'] = pd.to_numeric(data['top_cuisine']) # Защита. Преобразуем значения столбца в число

for y in range(0,len(data)):
    summa =0
    for x in cuisine:
        if  data[x][y] == 1:
            summa+=sort_dic.index(x)+1
    data.at[y,'top_cuisine'] = summa

## №3
Заполнение пустых ячеек столбца number_of_reviews. Пустые ячейки будем заполнять средним количеством отзывов по совокупной выборки конкретного ресторана.

In [ ]:

# Заполним сначала пустые ячейки столбца   reviews значением data['reviews'][0] = [[],[]]
data['reviews'] = data['reviews'].fillna(data['reviews'][0])

# Перевод значений столбца reviews в строку
for x in range(0, len(data)):
    data.at[x,'reviews'] = ''.join(data['reviews'][x])

# Так как встречаются варианты, когда количество отзывов нет, а в соседнем столбце reviews отзыв есть, скорректируем
# данные столбца number_of_reviews с 0 на 1
# 
for i in range(0,len(data)):
    if (data['number_of_reviews'][i] == 0) and (data['reviews'][i].count("'") !=0):
        data.at[i,'number_of_reviews'] = 1
        
# Группируем рестораны по id и вычислем среднее количество отзывов по каждой подгруппе        
g_df = pd.Series(data.groupby('restaurant_id')['number_of_reviews'].median())

# Корректируем данные столбца number_of_reviews изменяя значение 0 на среднее значение отзывов, если оно не равно 0.
for i in range(0,len(data)):
    if data['number_of_reviews'][i] ==0:
        x =  data['restaurant_id'][i]
        data.at[i,'number_of_reviews'] = round(g_df[x])

## №4
В столбце <b>reviews</b> содержится информация обзор отзывов посетителей.
Разнесён отзывы по столбцам.
Сформируем столбцы дат в которые формировались отзывы

In [ ]:
# Распарсим столбец на две составляющие, текст отзыва и даты, когда отзывы были сделаны. Текст оставим в столбце reviews, а для дат
# создадим столбцы date(и номер)
pattern = "[^a-z0-9/, ]+"
review = []# Список наименований столбцов, в которых будут занесены отзывы
dates = [] # Список наименований столбцов, в которых будут занесены даты отзывов
dat = []
txt = []
for x in range(0, len(data)):
    z = data['reviews'][x]
    z = re.sub('#',' ',z).lower().replace("\\', ",',').replace("'], ['",'#')\
                                 .replace("', '",'#').replace("'",'').replace(".",'')\
                                 .replace("-",' ').replace("[",'').replace("]",'')
    z = z.split('#')
    z = [y.strip() for y in z if y !=''] # Убираем пустые элементы из списка и удаляем крайние пробелы в элементах списка
    if len(z) == 2 and z[0] !=''and z[1] !='':
        txt.append(re.sub('[0-9/]+','',z[0]))
        dat.append(parser.parse(z[1]))
    elif len(z) == 4 and z[0] !='' and z[1] !=''and z[2] !='' and z[3] !='':
        txt.append(re.sub('[0-9/]+','',z[0]))
        txt.append(re.sub('[0-9/]+','',z[1]))
        dat.append(parser.parse(re.sub('[^0-9/]+','',z[2])))
        dat.append(parser.parse(re.sub('[^0-9/]+','',z[3])))
    da = [j for j in dat if j !='']    
    dl = len(da)
    if dl > 0:
        for d in range(0,dl):
            namd = 'date'+ str(d+1)
            data.at[x,namd] = da[d]
            if dates.count(namd) ==0:
                dates.append(namd)
    tx = [j for j in txt if j !='']
    dl = len(tx)
    if dl !=0:
        for d in range(0,dl):
            namt = 'review'+ str(d+1)
            data.at[x,namt] = tx[d]
            if review.count(namt) ==0:
                review.append(namt)
    dat.clear()
    txt.clear()

# Заполняем пустые ячейки столбцов отзывов нейтральным отзывом
for y in review:
    data[y] = data[y].fillna('neutral')

# Заполняем пустые ячейки столбцов дат 0
for y in dates:
    data[y] = data[y].fillna(0)
    
# Удаляем столбец reviews
data = data.drop(['reviews'], axis = 1)

# №5
Сформируем столбец разности дат.
Преобразуем даты озывов в дни от минимальной даты.

In [ ]:
# Формируем столбец разности дат отзывов в днях
data['days'] = 0
for y in range(0,len(data)):
    day1 = data['date1'][y]
    day2 = data['date2'][y]
    if day1 !=0 and day2 !=0:
        data.at[y,'days'] = abs((day1-day2).days)   
        
data['days'] = pd.to_numeric(data['days']) # Защита. Преобразуем значения столбца в число

In [ ]:
# Определяем минимальную дату отзыва.
data['date1'] = data['date1'].fillna(0)
data['date2'] = data['date2'].fillna(0)
data_min1 = min([y for y in data['date1'] if y !=0])
data_min2 = min([y for y in data['date2'] if y !=0])
if data_min2>data_min1:
    data_min = data_min1
else:
    data_min = data_min2
    
# Переводим столбцы дат отзывов в количество дней от минимальной даты
print(data_min)
for y in range(0,len(data)):
    if data['date1'][y] != 0:
        data.at[y,'date1'] = round((data['date1'][y] - data_min).days +1)
    else:
        data.at[y,'date1'] = 0        
for y in range(0,len(data)):
    if data['date2'][y] != 0:
        data.at[y,'date2'] = round((data['date2'][y] - data_min).days +1)
    else:
        data.at[y,'date2'] = 0
        
data['date1'] = pd.to_numeric(data['date1']) # Защита. Преобразуем значения столбца в число
data['date2'] = pd.to_numeric(data['date2']) # Защита. Преобразуем значения столбца в число

        
# Так как в ячейках находились значения 0 то, чтобы востановить исходные даты
#  необходимо восстанавливать с ячейки со значением 1

# Минимальная дата датасета 2004-04-21 00:00:00 указывается в сопроводительной справке, для востановления дат.

## №6
Преобразуем колонку price_range из разрядного представления цены в ячейках в столбцы
( 10, 100, 1000, 10000 максимальное значение ценового диапазона). Разнесём ценовые диапазоны по столбцам

In [ ]:
# Приведём значения ячеек к виду $, $$, $$$, $$$$
data['price'] = 0
for y in range(0,len(data)):
    if data['price_range'][y].count('$') == 5:
        data.at[y,'price'] = 3
    if data['price_range'][y].count('$') == 1:
        data.at[y,'price'] = 1
    if data['price_range'][y].count('$') == 4:
        data.at[y,'price'] = 4
    if data['price_range'][y].count('$') == 2:
        data.at[y,'price'] = 2

1. Группируем ресторанные цены по ресторанам и определяем среднее значение цены по каждой группе ресторанов.
2. Группируем ресторанные цены по городам и о пределяем среднее значение цены по городу

In [ ]:
price_rest = dict(data.groupby('restaurant_id')['price'].mean())
price_city = dict(data.groupby('city')['price'].mean())

#### Признак - доступности choice

Смысл признака choice в следующем. Так как в датасете представлены не просто 50000 ресторанов, а 11 с лишним  тысяч ресторанных сетей, то европейцы перемещаясь по Европе или туристы будут иметь опыт посещения той или иной ресторанной сети, и знакомы с её ценовым диапазоном, с которым они сталкивались ранее. При выборе ресторана куда пойти, потенциальный посетитель будет знакомиться с условиями обслуживания, в том числе и ценовой составляющей. Если по их опыту цена будет выше чем они привыкли, то вероятная мотивация отказаться от его посещения будет высокой и посетители будут ориентироваться на среднюю ценовую политику по всему городу. Если же цена будет ниже средней цены этой ресторанной сети, и если эта ресторанная сеть ранее устраивала посетителя, вероятность посещения будет высокой и признак choice по строке данного ресторана останется равным ресторанному. 

In [ ]:
for y in range(0,len(data)):
    if  data['price'][y] > price_rest[data['restaurant_id'][y]]:
        data.at[y,'choice'] = price_city[data['city'][y]]
    else:
        data.at[y,'choice'] = data['price'][y]

In [ ]:
data = pd.get_dummies(data, columns=[ 'price',], dummy_na=True) # Разносим столбец price_range
# Удаляем столбцы из списка price_range
data = data.drop(['price_range'], axis = 1)
data = data.drop(['price_nan'], axis = 1) # Удаляем столбцы price_range_nan

## №7

Разнесём наименование городов столбец - city, по столбцам.

In [ ]:
data = pd.get_dummies(data, columns=[ 'city',], dummy_na=True)
data = data.drop(['city_nan'], axis = 1)

## Окончательная обработка датасет
Определяем наименования столбцов подлежащих удалению.
Копируем обработанный датасет в новый датасет df_preproc и удаляем из нового датасета все столбцы значения которых не имеют числовой тип.

In [ ]:
# Формируем список наименований столбцов подлежащих удалению
list_col =[]
for x in data.columns:
    if data[x].dtype != 'float64' and data[x].dtype != 'int64' and data[x].dtype != 'uint8':
        list_col.append(x)
list_col

In [ ]:
df_preproc = data.copy()
df_preproc = df_preproc.drop(list_col, axis = 1)
df_preproc.info()

### Проведём стандартизацию и заполним нулями пропуски по всем переменным

In [ ]:

scaler = StandardScaler()
for i  in list(df_preproc.columns):
    if i not in ['rating','sample']:
        scaler.fit(df_preproc[[i]])
        df_preproc[i] = scaler.transform(df_preproc[[i]])
        if len(df_preproc[df_preproc[i].isna()]) < len(df_preproc):
            df_preproc[i] = df_preproc[i].fillna(0)

display(df_preproc.describe().head(8))

In [ ]:
df_corr = df_preproc.corr()
display(df_corr)

In [ ]:
# Выводим признаки, уровень корреляции между которыми выше 0.8
for x in df_corr.columns:
    for y in range(0,len(df_corr)):
        z = df_corr[x][y]
        if df_corr[x][y] > 0.8 and x != df_corr.columns[y]:            
            print(z,x,df_corr.columns[y])

In [ ]:
# Формируем список столбцов подлежащих удалению в связи с корреляционной зависимостью.
list_del = ['food_prices','city_London','types_cuisine']
#  -Удаляем food_prices, так как он входит в более расширенный признак living_index
#  -Удаляем city_London, так как часть признака tourist_people
#  -Удаляем types_cuisine, так как часть признака top_cuisine

# Удаляем столбцы из списка list_del
df_preproc = df_preproc.drop(list_del, axis = 1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values # наш таргет
X = train_data.drop(['rating'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
y_pred_rounded = np.round(y_pred*2)/2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

print('MAE с округлением результата:', metrics.mean_absolute_error(y_test, y_pred_rounded))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

### Вывод

Видим, что все введённые новые признаки вошли в список важных и оказывают влияние на рейтинг ресторана.
Учитывая, что из признаков исходного  датасета важным является лишь признак ранга - ranking, а все остальные сформированы отдельно, и относятся к социологическим, можно сделать выводы, что чем благоприятней общегородская социальная обстановка в городе и развита инфраструктура въездного туризма, тем выше посещаемость ресторана (независимо от типов кухонь их количества в одном ресторане, ....тип кухни так сказать на любителя).


#### Submission
готовим Submission на Kaggle.com

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()